# Beginner Tutorial: Recommender Systems in Python
from: https://www.datacamp.com/tutorial/recommender-systems-python

In [1]:
import pandas as pd
mm = pd.read_csv('data\\movies_metadata.csv', low_memory=False)
mm.info()
# 45466 rows of movies, with 24 columns of information.
# quite a few n/a values in: homepage, belongs_to_collection and tagline (others with minor occurance of N/A's too)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

# 1. Calc a Weighted Rating
We'll use this to calc a bayesian estimate between the items rating and the overall average rating weighted the number of votes 
$$
WR = \left( \frac{v}{v + m} \cdot R \right) + \left( \frac{m}{v + m} \cdot C \right)
$$
(thx to chatgpt for the markup)

WR = weighted rating <br>
v = number of votes per movie (provided) <br>
m = minimum votes required (our hyperparameter - we'll set this ourselves) <br>
R = average rating of the movie (provided)<br>
C =  mean rating across all movies (we'll calc this) <br>

In [2]:
# Getting C - average rating across all movies
C = mm['vote_average'].mean()
C

np.float64(5.618207215134185)

In [3]:
# Getting m - Deciding the min amount of votes required to be as the 90th percentile (cause the datacamp article does so)
m = mm['vote_count'].quantile(0.90)
print(m)

160.0


In [4]:
# extracting the qualified movies into seperate DF - about 10% of the total df
qm = mm.copy().loc[mm['vote_count']> m]
qm.shape


(4538, 24)

In [5]:
#defining function to return the weighted average 
def weighted_avg(df, m=m, C=C):
    v =  df['vote_count']
    R =  df['vote_average']
    return(v/(v+m) * R) + (m/(m+v)*C)

In [6]:
# adding the calculated weighted avg to new column 'score'
qm['score']  = qm.apply(weighted_avg, axis=1)
# sorting by new column
qm = qm.sort_values('score', ascending=False)
#print top 20 movies based on weighted avg
qm[['title','vote_count','vote_average','score']].head(20)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171
